In [1]:
import numpy as np
import pandas as pd
import ROOT
import subprocess
import glob
import os
import argparse

os.chdir("../input")

Welcome to JupyROOT 6.29/01


# Collect Results

In [2]:
def run_benchmark_gpu(files, n, m, w):
    n_pts = len(files)
    setup_times = np.zeros(n_pts)
    dev_st = np.zeros(n_pts)
    decomp_times = np.zeros(n_pts)
    dev_dt = np.zeros(n_pts)
    ratios = np.zeros(n_pts)
    sizes = np.zeros(n_pts)

    for i, f in enumerate(files):
        result = subprocess.run(
            [
                "../gpu_root_decomp",
                "-f",
                f,
                "-t",
                f.split(".")[-1],
                "-n",
                str(n),
                "-m",
                str(m),
                "-w",
                str(w),
            ],
            stdout=subprocess.PIPE,
        )

        output = result.stdout.decode("utf-8").split()
        setup_times[i], dev_st[i], decomp_times[i], dev_dt[i], ratios[i] = [
            float(o) for o in output[-7:-2]
        ]
        sizes[i] = int(f.split(".")[1]) 

#     print(setup_times, dev_st, decomp_times, dev_dt, ratios, sizes)
    return setup_times, dev_st, decomp_times, dev_dt, ratios, sizes

In [3]:
def run_benchmark_cpu(files, n, m, w, c=16):
    n_pts = len(files)
    decomp_times = np.zeros(n_pts)
    dev_dt = np.zeros(n_pts)
    ratios = np.zeros(n_pts)
    sizes = np.zeros(n_pts)

    for i, f in enumerate(files):
        result = subprocess.run(
            [
                "../cpu_root_decomp",
                "-f",
                f,
                "-s",
                f.split(".")[1],
                "-n",
                str(n),
                "-m",
                str(m),
                "-c",
                str(c),
                "-w",
                str(w),
            ],
            stdout=subprocess.PIPE,
        )
        output = result.stdout.decode("utf-8").split()
        decomp_times[i] = float(output[-8])
        dev_dt[i] = float(output[-4])
        ratios[i] = float(output[-1])
        sizes[i] = int(f.split(".")[1]) 

#     print(decomp_times, dev_dt, sizes)
    return decomp_times, dev_dt, ratios, sizes

In [4]:
def sort_lists(to_sort, sort_idx):
    return [l[sort_idx] for l in to_sort]

# Same batch size, different chunk size

In [42]:
def plot_against_size(c, cpu_results, gpu_results, title):
    cpu_decomp_times, cpu_dev_dt, _, _ = cpu_results
    (
        gpu_setup_times,
        gpu_dev_st,
        gpu_decomp_times,
        gpu_dev_dt,
        ratios,
        sizes,
    ) = gpu_results

    n_pts = len(sizes)
    zeroes = np.zeros(n_pts, dtype=float)

    mg = ROOT.TMultiGraph()
    g1 = ROOT.TGraphErrors(n_pts, sizes, gpu_setup_times, zeroes, gpu_dev_st)
    g1.SetLineColor(ROOT.kRed)

    g2 = ROOT.TGraphErrors(n_pts, sizes, gpu_decomp_times, zeroes, gpu_dev_dt)
    g2.SetLineColor(ROOT.kBlue)

    g3 = ROOT.TGraphErrors(n_pts, sizes, cpu_decomp_times, zeroes, cpu_dev_dt)
    
    # c.SetLogx()
    c.SetRightMargin(0.32)
    mg.Add(g1)
    mg.Add(g2)
    mg.Add(g3)
    
    mg.SetTitle(title)
    mg.GetXaxis().SetTitle("Decompressed size (B)")
    mg.GetXaxis().SetLimits(0, np.max(sizes) + np.min(sizes))
    mg.GetYaxis().SetTitle("Time (ms)")
    mg.GetYaxis().SetLimits(0, np.max(gpu_decomp_times) + 10)
    mg.DrawClone("AL")
    
    l = ROOT.TLegend(0.687, 0.7, 0.99, 0.9)
    l.AddEntry(g1, "GPU - Setup")
    l.AddEntry(g2, "GPU - Decompression")
    l.AddEntry(g3, "CPU - Decompression")
    l.SetTextSize(0.03)
    l.DrawClone()

In [6]:
%jsroot on
n = 10
w = 10
m = 15
files = glob.glob("low_compression*.root.zstd")
gpu_results = run_benchmark_gpu(files, n, m, w)
cpu_results = run_benchmark_cpu(files, n, m, w)
cpu_results = sort_lists(cpu_results, np.argsort(cpu_results[-1])) # Sort by size
gpu_results = sort_lists(gpu_results, np.argsort(gpu_results[-1])) # Sort by size
title = f"Decompression of ROOT compressed files with average ratio: {np.mean(gpu_results[-2])}"
c = ROOT.TCanvas("c1", title)
plot_against_size(c, cpu_results, gpu_results, title)
c.DrawClone()

In [7]:
%jsroot on
n = 10
w = 10
m = 15
files = glob.glob("mid_compression*.root.zstd")
gpu_results = run_benchmark_gpu(files, n, m, w)
cpu_results = run_benchmark_cpu(files, n, m, w)
cpu_results = sort_lists(cpu_results, np.argsort(cpu_results[-1])) # Sort by size
gpu_results = sort_lists(gpu_results, np.argsort(gpu_results[-1])) # Sort by size
title = f"Decompression of ROOT compressed files with average ratio: {np.mean(gpu_results[-2])}"
c = ROOT.TCanvas("c1", title)
plot_against_size(c, cpu_results, gpu_results, title)
c.DrawClone()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [43]:
%jsroot on
n = 10
w = 10
m = 15
files = glob.glob("max_compression*.root.zstd")

gpu_results = run_benchmark_gpu(files, n, m, w)
cpu_results = run_benchmark_cpu(files, n, m, w)
cpu_results = sort_lists(cpu_results, np.argsort(cpu_results[-1])) # Sort by size
gpu_results = sort_lists(gpu_results, np.argsort(gpu_results[-1])) # Sort by size
title = f"Decompression of ROOT compressed files with average ratio: {np.mean(gpu_results[-2])}"
c = ROOT.TCanvas("c1", title)
plot_against_size(c, cpu_results, gpu_results, title)
c.DrawClone()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


Combined attempt

In [9]:
%jsroot on

def plot_against_size(c, cpu_results, gpu_results, title=""):
    cpu_decomp_times, cpu_dev_dt, _, _ , _= cpu_results
    (
        gpu_setup_times,
        gpu_dev_st,
        gpu_decomp_times,
        gpu_dev_dt,
        ratios,
        sizes,
    ) = gpu_results

    n_pts = len(sizes)
    zeroes = np.zeros(n_pts, dtype=float)

    mg = ROOT.TMultiGraph()
    g1 = ROOT.TGraphErrors(n_pts, sizes, gpu_setup_times, zeroes, gpu_dev_st)
    g1.SetLineColor(ROOT.kRed)

    g2 = ROOT.TGraphErrors(n_pts, sizes, gpu_decomp_times, zeroes, gpu_dev_dt)
    g2.SetLineColor(ROOT.kBlue)

    g3 = ROOT.TGraphErrors(n_pts, sizes, cpu_decomp_times, zeroes, cpu_dev_dt)
    
    mg.Add(g1)
    mg.Add(g2)
    mg.Add(g3)
    
    mg.SetTitle(title)
    mg.GetXaxis().SetTitle("Decompressed size (B)")
    mg.GetXaxis().SetLimits(0, np.max(sizes) + np.min(sizes))
    mg.GetYaxis().SetTitle("Time (ms)")
    mg.GetYaxis().SetLimits(0, np.max(gpu_decomp_times) + 10)
    mg.DrawClone("AL")
    return l
                     
c = ROOT.TCanvas("c1")
c.Divide(3,1);

c.cd(1)
n = 10
w = 10
m = 100
files = glob.glob("low_compression*.root.zstd")
gpu_results = run_benchmark_gpu(files, n, m, w)
cpu_results = run_benchmark_cpu(files, n, m, w)
cpu_results = sort_lists(cpu_results, np.argsort(cpu_results[-1])) # Sort by size
gpu_results = sort_lists(gpu_results, np.argsort(gpu_results[-1])) # Sort by size
title = f"Decompression of ROOT compressed files with average ratio: {np.mean(gpu_results[-2])}"
plot_against_size(c, cpu_results, gpu_results, title)

c.cd(2)
n = 10
w = 10
m = 100
files = glob.glob("mid_compression*.root.zstd")
gpu_results = run_benchmark_gpu(files, n, m, w)
cpu_results = run_benchmark_cpu(files, n, m, w)
cpu_results = sort_lists(cpu_results, np.argsort(cpu_results[-1])) # Sort by size
gpu_results = sort_lists(gpu_results, np.argsort(gpu_results[-1])) # Sort by size
title = f"Decompression of ROOT compressed files with average ratio: {np.mean(gpu_results[-2])}"
plot_against_size(c, cpu_results, gpu_results, title)

c.cd(3)
n = 10
w = 10
m = 100
files = glob.glob("max_compression*.root.zstd")
gpu_results = run_benchmark_gpu(files, n, m, w)
cpu_results = run_benchmark_cpu(files, n, m, w)
l = plot_against_size(c, cpu_results, gpu_results, title)

l.SetTextSize(0.03)
l.Draw()                     
c.cd()
c.Draw()

ValueError: not enough values to unpack (expected 5, got 4)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


# Same chunk size, different batch size

In [36]:
def plot_scsdbs(c, n, w, m, file, mode="both"):
    %jsroot on
    cpu_r = np.array([[],[],[],[],[]] )
    gpu_r = np.array([[],[],[],[],[],[],[]] )
    for n_chunks in m:
        cpu_results = *run_benchmark_cpu(file, n, n_chunks, w), np.array([n_chunks])
        gpu_results = *run_benchmark_gpu(file, n, n_chunks, w), np.array([n_chunks])
        cpu_results = sort_lists(cpu_results, np.argsort(cpu_results[-1])) # Sort by size
        gpu_results = sort_lists(gpu_results, np.argsort(gpu_results[-1])) # Sort by size
        cpu_r = np.hstack((cpu_r, cpu_results))
        gpu_r = np.hstack((gpu_r, gpu_results))

    title = f"Decompression of ROOT compressed file with chunk size: {file[0].split('.')[1]}"
    cpu_decomp_times, cpu_dev_dt, _, _, _ = cpu_r
    (
        gpu_setup_times,
        gpu_dev_st,
        gpu_decomp_times,
        gpu_dev_dt,
        ratios,
        sizes,
        nchunks
    ) = gpu_r

    nchunks = nchunks.astype(float)
    n_pts = len(sizes)
    zeroes = np.zeros(n_pts, dtype=float)

    mg = ROOT.TMultiGraph()
    c.SetRightMargin(0.32)
    l = ROOT.TLegend(0.687, 0.7, 0.99, 0.9)
    
    g1 = ROOT.TGraphErrors(n_pts, nchunks, gpu_setup_times, zeroes, gpu_dev_st)
    g1.SetLineColor(ROOT.kRed)

    g2 = ROOT.TGraphErrors(n_pts, nchunks, gpu_decomp_times, zeroes, gpu_dev_dt)
    g2.SetLineColor(ROOT.kBlue)
    
    if mode == "both" or mode == "cpu":
        g3 = ROOT.TGraphErrors(n_pts, nchunks, cpu_decomp_times, zeroes, cpu_dev_dt)
        mg.Add(g3)
        l.AddEntry(g3, "CPU - Decompression")

    mg.Add(g1)
    mg.Add(g2)

    mg.SetTitle(title)
    mg.GetXaxis().SetTitle("Number of chunks")
    # mg.GetXaxis().SetLimits(0, np.max(nchunks) + np.min(nchunks))
    mg.GetYaxis().SetTitle("Time (ms)")
    # mg.GetYaxis().SetLimits(0, np.max(gpu_decomp_times) + 10)
    mg.DrawClone("AL")

    l.AddEntry(g1, "GPU - Setup")
    l.AddEntry(g2, "GPU - Decompression")
    l.SetTextSize(0.03)
    l.DrawClone()

n = 10
w = 10
m = [1, 2, 4, 8, 16, 32, 64, 128]
file = glob.glob("mid_compression.4000.root.zstd")
c = ROOT.TCanvas("c11", title)
plot_scsdbs(c,n,w,m,file)
c.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c11


In [31]:
n = 10
w = 10
m = [1, 2, 4, 8, 16, 32, 64, 128]
file = glob.glob("mid_compression.32000.root.zstd")
c = ROOT.TCanvas("c12", title)
plot_scsdbs(c,n,w,m,file)
c.Draw()

In [32]:
n = 10
w = 10
m = [1, 2, 4, 8, 16, 32, 64, 128]
file = glob.glob("mid_compression.128000.root.zstd")
c = ROOT.TCanvas("c13", title)
plot_scsdbs(c,n,w,m,file)
c.Draw()

In [39]:
%jsroot on

n = 10
w = 10
m = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
file = glob.glob("mid_compression.128000.root.zstd")
c = ROOT.TCanvas("c15", title)
plot_scsdbs(c,n,w,m,file,mode="gpu")
c.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c15


In [40]:
%jsroot on

n = 10
w = 10
m = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
file = glob.glob("mid_compression.4000.root.zstd")
c = ROOT.TCanvas("c15", title)
plot_scsdbs(c,n,w,m,file,mode="gpu")
c.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c15


# Plot Results 3D

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

n = 10
w = 10
m = [1, 2, 4]
files = glob.glob("*.root.zstd")
n_pts = len(files) 

ax = plt.figure().add_subplot(projection='3d')

# gpu_r = np.array([])
# cpu_r = np.array([])
for n_chunks in m:
    gpu_results = run_benchmark_gpu(files, n, n_chunks, w)
    cpu_results = run_benchmark_cpu(files, n, n_chunks, w)
    cpu_results = sort_lists(cpu_results, np.argsort(cpu_results[-1])) # Sort by size
    gpu_results = sort_lists(gpu_results, np.argsort(gpu_results[-1])) # Sort by size
    cpu_decomp_times, cpu_dev_dt = cpu_results
    (
        gpu_setup_times,
        gpu_dev_st,
        gpu_decomp_times,
        gpu_dev_dt,
        ratios,
        sizes,
    ) = gpu_results
    
    ax.scatter(np.repeat(n_chunks, n_pts), sizes, gpu_setup_times, color="r")
    ax.scatter(np.repeat(n_chunks, n_pts), sizes, gpu_decomp_times, color="b")
    ax.scatter(np.repeat(n_chunks, n_pts), sizes, cpu_decomp_times, color="k")
    
plt.show()

## File size

In [ ]:
n = 10
w = 10
m = 100
files = glob.glob("*.root.zstd")

c = ROOT.TCanvas("c1", "Decompression of ROOT compressed files")
gpu_results = run_benchmark_gpu(files, n, m, w)
cpu_results = run_benchmark_cpu(files, n, m, w)
cpu_results = sort_lists(cpu_results, np.argsort(cpu_results[-1])) # Sort by size
gpu_results = sort_lists(gpu_results, np.argsort(gpu_results[-1])) # Sort by size
cpu_decomp_times, cpu_dev_dt = cpu_results
(
    gpu_setup_times,
    gpu_dev_st,
    gpu_decomp_times,
    gpu_dev_dt,
    ratios,
    sizes,
) = gpu_results
sizes *= m

n_pts = len(sizes)
zeroes = np.zeros(n_pts, dtype=float)

mg = ROOT.TMultiGraph()
g1 = ROOT.TGraphErrors(n_pts, sizes, gpu_setup_times, zeroes, gpu_dev_st)
g1.SetLineColor(ROOT.kRed)

g2 = ROOT.TGraphErrors(n_pts, sizes, gpu_decomp_times, zeroes, gpu_dev_dt)
g2.SetLineColor(ROOT.kBlue)

g3 = ROOT.TGraphErrors(n_pts, sizes, cpu_decomp_times, zeroes, cpu_dev_dt)

c.SetRightMargin(0.32)
mg.Add(g1)
mg.Add(g2)
mg.Add(g3)

mg.GetXaxis().SetTitle("Decompressed size (B)")
mg.GetXaxis().SetLimits(0, np.max(sizes) + np.min(sizes))
mg.GetYaxis().SetTitle("Time (ms)")
mg.GetYaxis().SetLimits(0, np.max(gpu_decomp_times) + 10)
mg.DrawClone("AL")

l = ROOT.TLegend(0.687, 0.7, 0.99, 0.9)
l.AddEntry(g1, "GPU - Setup")
l.AddEntry(g2, "GPU - Decompression")
l.AddEntry(g3, "CPU - Decompression")
l.SetTextSize(0.03)
l.DrawClone()
c.Draw()

## Size separated by page sizes and chunks

In [ ]:
%jsroot on
n = 10
w = 10
m = [1, 2, 4]
files = glob.glob("*.root.zstd")
n_pts = len(files) * len(m)

c = ROOT.TCanvas("c2", "Decompression of ROOT compressed files")
g1 = ROOT.TH3F()
g1.SetTitle("Decompression of ROOT compressed files")
g2 = ROOT.TH3F()
# g3 = ROOT.TGraph2D(n_pts)

for n_chunks in m:
    gpu_results = run_benchmark_gpu(files, n, n_chunks, w)
    cpu_results = run_benchmark_cpu(files, n, n_chunks, w)
    cpu_results = sort_lists(cpu_results, np.argsort(cpu_results[-1])) # Sort by size
    gpu_results = sort_lists(gpu_results, np.argsort(gpu_results[-1])) # Sort by size
    cpu_decomp_times, cpu_dev_dt = cpu_results
    (
        gpu_setup_times,
        gpu_dev_st,
        gpu_decomp_times,
        gpu_dev_dt,
        ratios,
        sizes,
    ) = gpu_results
    
    for p in range(len(sizes)):
        g1.Fill(n_chunks, sizes[p], gpu_setup_times[p])
        g2.Fill(n_chunks, sizes[p], gpu_decomp_times[p])
#         g3.AddPoint(n_chunks, sizes[p], cpu_decomp_times[p])

c.SetRightMargin(0.3)
g1.SetLineColor(ROOT.kRed)
g2.SetLineColor(ROOT.kBlue)

g1.DrawClone()

g1.GetXaxis().SetTitle("Number of chunks")
g1.GetXaxis().SetTitleOffset(0.5)
# g1.GetXaxis().SetLimits(0, np.max(sizes) + np.min(sizes))
g1.GetYaxis().SetTitle("Chunk size (B)")
g1.GetXaxis().SetTitleOffset(0.5)
# g1.GetYaxis().SetLimits(0, np.max(gpu_decomp_times) + 10)
g1.GetZaxis().SetTitle("Time (ms)")
g1.GetXaxis().SetTitleOffset(0.5)

# g1.GetYaxis().SetLimits(0, np.max(gpu_decomp_times) + 10)

g2.DrawClone("same")
# g3.DrawClone("same p0")


l = ROOT.TLegend(0.687, 0.7, 0.99, 0.9)
l.AddEntry(g1, "GPU - Setup")
l.AddEntry(g2, "GPU - Decompression")
# l.AddEntry(g3, "CPU - Decompression")
l.SetTextSize(0.03)
l.SetMargin(0.25)
l.DrawClone()
c.Draw()